In [11]:
import pandas as pd
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join('..')))

ENV_NAME = "MOHalfCheetahDR-v5" # CHANGE THIS TO THE NAME OF THE ENVIRONMENT
REWARD_DIM = 2 # CHANGE THIS TO THE NUMBER OF OBJECTIVES IN THE ENVIRONMENT

### Get Normalization Params

In [12]:
from evaluation import get_eval_params

normalization_data = get_eval_params(ENV_NAME)['normalization']
normalization_data

{'MOHalfCheetahDefault-v5': {0: [0, 836], 1: [-353, -3.4]},
 'MOHalfCheetahLight-v5': {0: [0, 1060], 1: [-345, -4.3]},
 'MOHalfCheetahHeavy-v5': {0: [0, 227], 1: [-379, -4.3]},
 'MOHalfCheetahSlippery-v5': {0: [0, 791], 1: [-411, -3.6]},
 'MOHalfCheetahHard-v5': {0: [0, 511], 1: [-417, -3.8]}}

### Normalize Front and Calculate Normalized Hypervolume and EUM

Import helpers

In [13]:
import numpy as np

sys.path.append(os.path.abspath(os.path.join('../..')))

def get_normalized_vec_returns(all_vec_returns, minmax_range):
    minmax_array = np.array([minmax_range[i] for i in range(all_vec_returns.shape[-1])])
    min_vals = minmax_array[:, 0].reshape(1, 1, -1) # reshape to (1, 1, n_objectives) for broadcasting
    max_vals = minmax_array[:, 1].reshape(1, 1, -1)

    clipped_vec_returns = np.clip(all_vec_returns, min_vals, max_vals) # broadcasted clipping
    
    # Normalize
    normalized_vec_returns = (clipped_vec_returns - min_vals) / (max_vals - min_vals)
    
    return normalized_vec_returns

In [14]:
from mo_utils.performance_indicators import hypervolume, expected_utility
from mo_utils.weights import equally_spaced_weights

NUM_WEIGHTS = 100
EVAL_WEIGHTS = equally_spaced_weights(REWARD_DIM, NUM_WEIGHTS) 

EVAL_WEIGHTS[0:5]

[array([0., 1.]),
 array([0.00990285, 0.99009715]),
 array([0.01981506, 0.98018494]),
 array([0.02981648, 0.97018352]),
 array([0.03979621, 0.96020379])]

Normalize the fronts and calculate normalized hypervolume and EUM

In [15]:
from helpers.utils import ENVIRONMENTS_MAP, ALGORITHMS

FRONT = "eval/discounted_front" # change this if you are using undiscounted front, but why?
file_path = f"data/{FRONT}/{ENV_NAME}/"
scores_save_path = f"data/scores/{ENV_NAME}/"

SEEDS = [5,26,47,76,92]

# Load the data
for algo in ALGORITHMS:
    for seed in SEEDS:
        normalized_hypervolumes = []
        normalized_eums = []
        for env in ENVIRONMENTS_MAP[ENV_NAME]:
            min_max_ranges = normalization_data[env]
            file = f"{file_path}/{algo}/seed_{seed}/{env}.csv"
            assert os.path.exists(file), f"File {file} does not exist"
            data = pd.read_csv(file)
            # Convert dataframe to numpy array of vectors
            data_array = data.to_numpy()
            normalized_front = get_normalized_vec_returns(data_array, min_max_ranges)

            normalized_hypervolumes.append(hypervolume(np.zeros(REWARD_DIM), normalized_front[0]))
            normalized_eums.append(expected_utility(normalized_front[0], weights_set=EVAL_WEIGHTS))

        data = {f"normalized_hypervolume/{env}": [normalized_hypervolumes[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])}
        data.update({f"normalized_eum/{env}": [normalized_eums[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])})
        df = pd.DataFrame(data)
        os.makedirs(f"{scores_save_path}/{algo}/", exist_ok=True)
        df.to_csv(f"{scores_save_path}/{algo}/seed_{seed}.csv", index=False)
            